# 1 Speed up Your Python Codes

Python is one of the most popular programming languages among developers due to its community support, amazing libraries, wide usage in Machine Learning and Big Data, and easy syntax. However, as an interpreted language, Python has one significant drawback, which is its slow speed. 

## 1.1 Rules to speed up your Python codes

- Decrease the use of `for loop`
> this is the RULE NO.1.
- Use the List Comprehensions
> ```[i for i in range (1, 1000) if i%3 == 0]``` is faster than `for loop + append()` 
- Use the Built-In Functions
> Such as ```map(), sum(), mean(), median(), max(), min()```
- Function Calls Are Expensive
> It is better to iterate inside a function than to iterate and call a function each iteration.
- Use Numpy and Scipy
> This is RULE NO.2. Search a solution in Numpy and Scipy first before you code it up.
- Avoid Lazy Module Importing
>```python
>from math import sqrt # but not from math import *
>val = sqrt(60)
>```
- Be Careful with Bulky Libraries
> Reduce the number and size of dependencies, keep your code simple.
- Avoid Global Variables
> Local Variables lead to less memory usage and higher performance, it is simply best to avoid global variables when possible.
- Adopt Proper Algorithms and Data Structures
> - Algorithms are like science, while optimization is like technique. When the current technique can not take you to a galaxy far far away, try to find a worm whole.
> - Data structure provides the right way to organize information in the digital space. Consider using `FITS`, `HDF5`, `Pandas`, and so on, before start your astronomical research.

# 1.2 Ways to Speed Up your Python code

- Use Exsiting Packages: Numpy/Scipy/Numba
- Python-C/C++/Fortran Integration
- Mutiprocessing/MPI4py/Ctypes+OpenMP

# 2 Practice
`Simpson Integration with Python`

\begin{equation}
\int^{x_{i+1}}_{x_{i-1}} f(x)~dx = \frac{h}{3} \left( f(x_{i−1})+4 f(x_i)+f(x_{i+1}) \right)+O(h^5) 
\end{equation}

### 2.1 Basic Approaches (with Numpy, Scipy, and Numba)

- Python Only

In [77]:
%%time

from math import sin, pi

def func_python(x): 
    return sin(x)

def simpson_python(f,a,b,N=100000000): # N show be even or odd?
    dx = (b-a)/N
    S = 0
    for i in range(1, N, 2):
        x_m1 = a+(i-1)*dx
        x_i =  a+(i)*dx
        x_p1 = a+(i+1)*dx
        y_m1 = f(x_m1)
        y_i = f(x_i)
        y_p1 = f(x_p1)
        S += dx/3 * (y_m1 + 4.0*y_i + y_p1)
    return S

simpson_python(func_python, 0.0, pi/2)

CPU times: user 27.7 s, sys: 147 ms, total: 27.9 s
Wall time: 28 s


0.9999999999997551

- Numpy Only

In [79]:
%%time

import numpy as np

def func_numpy(x): 
    return np.sin(x)

def simpson_numpy(f,a,b,N=100000000):
    dx = (b-a)/N
    x = np.linspace(a,b,N+1)
    y = f(x)
    S = dx/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])
    return S

simpson_numpy(func_numpy, 0.0, np.pi/2)

CPU times: user 1.4 s, sys: 503 ms, total: 1.9 s
Wall time: 1.91 s


0.9999999999999988

- Numpy + scipy.integrate.simps

In [82]:
%%time

# use arrays instead of calling functions
import numpy as np

N_arr = 100000001; # is this number even or odd? 
a_arr = 0.0; 
b_arr = np.pi/2;
x_arr = np.linspace(a_arr, b_arr, N_arr)
y_arr = func_numpy(x_arr)

from scipy import integrate as spi

print(spi.simps(y_arr, x_arr))

0.9999999999999976
CPU times: user 2.74 s, sys: 1.4 s, total: 4.14 s
Wall time: 4.16 s


* Numpy + Numpy

In [83]:
%%time

import numpy as np

N_arr = 100000001;
a_arr = 0.0; 
b_arr = np.pi/2;
x_arr = np.linspace(a_arr, b_arr, N_arr)
y_arr = func_numpy(x_arr)

def simpson_numpy_arr(y, x):
    S = (x[1]-x[0])/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])
    return S

print(simpson_numpy_arr(y_arr, x_arr))

0.9999999999999988
CPU times: user 1.46 s, sys: 441 ms, total: 1.9 s
Wall time: 1.9 s


- Numpy + Numba

In [84]:
%%time

import numpy as np

N_arr = 100000001;
a_arr = 0.0; 
b_arr = np.pi/2;
x_arr = np.linspace(a_arr, b_arr, N_arr)
y_arr = func_numpy(x_arr)

from numba import jit

@jit(nopython=True)
def simpson_numba(y, x):
    N = len(x)
    dx = x[1] - x[0]
    S = 0
    for i in range(1, N, 2):
        S += dx/3 * (y[i-1] + 4.0*y[i] + y[i+1])
    return S

print(simpson_numba(y_arr, x_arr))

0.9999999999997551
CPU times: user 1.29 s, sys: 364 ms, total: 1.66 s
Wall time: 1.66 s


***The fastest way is `Numpy + Numba`, but the easiest way is `Numpy + Numpy`***

### 2.2 Advanced Approaches (with C-integrations and OpenMP)

- Ctypes in Python
    1. compose a standard C code
        ```C
            #include <stdio.h>

            double simpson_c(double * y, double * x, long N)
            {
                double dx = x[1] - x[0];
                double S = 0.0;
                long i;
                for (i=0;i<N;i+=2)
                {
                    S += dx/3. * (y[i] + 4.0*y[i+1] + y[i+2]);
                }
                return S;
            }
        ```
    2. compile it to a shared library
        ```bash
        $ gcc-11 -O2 --shared example.c -o example.so -Wall
        ```
    3. import `ctypes` and load the C library
    4. create a "header" for the C function
    5. create a Python function for the C function

In [85]:
# import ctypes and load the C library
import ctypes as ct
ctypes_example = ct.CDLL("./example.so")

In [86]:
# create a "header" for the C function
ctypes_example.simpson_c.argtypes = [np.ctypeslib.ndpointer(dtype = ct.c_double), \
                                     np.ctypeslib.ndpointer(dtype = ct.c_double), \
                                     ct.c_long]
ctypes_example.simpson_c.restype  = ct.c_double

# create a Python function for the C function
def call_simpson_c(y, x):
    N_in = ct.c_long(len(x))
    y_in = np.array(y, dtype=ct.c_double)
    x_in = np.array(x, dtype=ct.c_double)
    res = ctypes_example.simpson_c(y_in, x_in, N_in)
    return res

In [87]:
%%time

print(call_simpson_c(y_arr, x_arr))

1.000000005235743
CPU times: user 494 ms, sys: 422 ms, total: 915 ms
Wall time: 919 ms


- Ctypes in Python with OpenMP
    1. compose a standard C code
        ```C
            #include <stdio.h>
            #include <omp.h>

            double simpson_c_omp(double * y, double * x, long N, int nThreads)
            {
                double dx = x[1] - x[0];
                double S;
                S = 0.0;
                long i;
                #pragma omp parallel \
                num_threads(nThreads) \
                shared(y, x, dx, N, S) \
                private(i)
                {
                    double partial_S = 0.0;
                    #pragma omp for
                    for (i=0;i<(N-1)/2;i++)
                    {
                        partial_S += dx/3. * (y[2*i] + 4.0*y[2*i+1] + y[2*i+2]);
                    }
                    #pragma omp critical
                    S += partial_S;
                }
                return S;
            }
        ```
    2. compile it to a shared library
        ```bash
        $ gcc-11 -O2 --shared example.c -o example.so -Wall -fopenmp
        ```
    3. import `ctypes` and load the C library 
    4. create a "header" for the C function
    5. create a Python function for the C function

In [88]:
# import ctypes and load the C library
import ctypes as ct
ctypes_example = ct.CDLL("./example.so")

In [89]:
# create a "header" for the C function
ctypes_example.simpson_c_omp.argtypes = [np.ctypeslib.ndpointer(dtype = ct.c_double), \
                                         np.ctypeslib.ndpointer(dtype = ct.c_double), \
                                         ct.c_long, ct.c_int]
ctypes_example.simpson_c_omp.restype  = ct.c_double
# create a Python function for the C function
def call_simpson_c_omp(y, x, nThreads=4):
    N_in = ct.c_long(len(x))
    y_in = np.array(y, dtype=ct.c_double)
    x_in = np.array(x, dtype=ct.c_double)
    nThreads_in = ct.c_int(nThreads)
    res = ctypes_example.simpson_c_omp(y_in, x_in, N_in, nThreads_in)
    return res

In [90]:
%%time

print(call_simpson_c_omp(y_arr, x_arr, nThreads=8))

1.0000000000000182
CPU times: user 693 ms, sys: 413 ms, total: 1.11 s
Wall time: 861 ms


<font size='10' color='red'>However, at last, one way beyond them all...so please keep RULE NO.2 in mind</font> 

In [92]:
%%time

import numpy as np
from scipy import integrate as spi
spi.quad(np.sin, 0, np.pi/2)

CPU times: user 88 µs, sys: 2 µs, total: 90 µs
Wall time: 95.1 µs


(0.9999999999999999, 1.1102230246251564e-14)

### * 2.3 Superior Approaches (with Multiprocessing and MPI4py)

1. Multiprocessing (https://docs.python.org/3/library/multiprocessing.html)
2. MPI4py (https://mpi4py.readthedocs.io/en/stable/)
3. Other fancy approaches:
    - pyCuda (https://documen.tician.de/pycuda/)
    - pyOpenCL (https://documen.tician.de/pyopencl/)
    - Numba (Pro) (http://numba.pydata.org/)

### Multiprocessing Example 1

```python
import numpy as np                                                                              
from multiprocessing import Pool, cpu_count
 
def monte_carlo_pi_part(n):
    count = 0
    for i in range(int(n)):
        x=np.random.random()
        y=np.random.random()
        if x*x + y*y <= 1:
            count=count+1
    return count

if __name__ == '__main__':
    cores = 4
    print('You have %d CPUs, and you use %d cores for the calculation'%(cpu_count(), cores))
    n = 10000000
    part_count=[n/cores for i in range(cores)] # not accurate, can you improve it?
    with Pool(cores) as p:
        count=p.map(monte_carlo_pi_part, part_count)
 
    print("Esitmated value of Pi:: ", sum(count)/(n*1.0)*4  )
```

### Multiprocessing Example 2

```python
import numpy as np
from multiprocessing import Pool
                 
def simpson_tmp_arr(data_pairs):
    y = data_pairs[0]
    x = data_pairs[1]
    N = int(len(y))
    dx = x[1] - x[0]
    S = 0        
    for i in range(1, N, 2):
        S += dx/3 * (y[i-1] + 4.0*y[i] + y[i+1])
    return S     
                 
if __name__ == '__main__':
    N_arr = 100000004; # how to define this number properly?
    a_arr = 0.0; 
    b_arr = np.pi/2;
    x_arr = np.linspace(a_arr, b_arr, N_arr)
    y_arr = np.sin(x_arr)
                 
    cores = 4    
                 
    y_parts = np.array_split(y_arr, cores)
    x_parts = np.array_split(x_arr, cores)
                 
    pairs_parts = [[y_i, x_i] for [y_i, x_i] in zip(y_parts, x_parts)]
                 
    with Pool(cores) as p:
        parts = p.map(simpson_tmp_arr, pairs_parts)
                 
    print("Result = %.20f"%(np.sum(parts)))  
```

### Mpi4py Example 1

>```bash
>(base) [nanli@tulip ~]$ time mpirun -np 4 python mpi4py_integrate_arrays.py
>
>With n =  100000001 bins, 
>integral of sine(x) from 0.0 to 1.5707963267948966 = 1.0000000000000338 
>real    0m13.090s
>user    0m48.931s
>sys     0m2.418s
>```

```python
import numpy as np

def simpson_local_arr(y, x, i_start, i_end):
    dx = x[1] - x[0]
    S = 0
    if i_end >= len(x):
        i_end = int(len(x))
    for i in range(i_start, i_end, 2):
        S += dx/3 * (y[i-1] + 4.0*y[i] + y[i+1])
    return S

from mpi4py import MPI
if __name__ == '__main__':
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    dest=0
    total=-1.0

    N_arr = 100000001; # is this number even or odd? 
    a_arr = 0.0; 
    b_arr = np.pi/2;
    x_arr = np.linspace(a_arr, b_arr, N_arr)
    y_arr = np.sin(x_arr)

    local_n = int(N_arr/size)
    i_s = 1+rank*local_n
    i_e = i_s + local_n
    
    integral = simpson_local_arr(y_arr, x_arr, i_s, i_e)

    if rank == 0:
        total = integral
        for source in range(1,size):
            integral = comm.recv(source=source)
            print("PE ", rank, "<-", source, ",", integral, "\n")
            total = total + integral
    else:
        print("PE ", rank, "->", dest, ",", integral, "\n")
        comm.send(integral, dest=0)

    if (rank == 0):
        print("\n")
        print("With n = ", N_arr, "bins, \n")
        print("integral of sine(x) from", a_arr, "to", b_arr, "=", total, "\n")
    MPI.Finalize
```

### Mpi4py Example 2
>
>```bash
>(base) [nanli@tulip ~]$ time mpirun -np 4 python mpi4py_Sum_arrays.py 
>
>S =  [1.00000001]
>
>real    0m22.751s
>user    1m24.098s
>sys     0m5.683s
>```
>

```python
import numpy as np

def simpson_local_arr(y, x, i_start, i_end):
    dx = x[1] - x[0]
    S = 0
    if i_end >= len(x):
        i_end = int(len(x))
    for i in range(i_start, i_end, 2):
        S += dx/3 * (y[i-1] + 4.0*y[i] + y[i+1])
    return S

from mpi4py import MPI
if __name__ == '__main__':
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    
    dest=0
    total=-1.0

    N_arr = 100000001; # is this number even or odd? 
    a_arr = 0.0; 
    b_arr = np.pi/2;
    x_arr = np.linspace(a_arr, b_arr, N_arr)
    y_arr = np.sin(x_arr)

    comm.Bcast([y_arr, MPI.DOUBLE], root=0)
    comm.Bcast([x_arr, MPI.DOUBLE], root=0)

    if rank == 0:
        sendbuf = np.arange(N_arr)
        ave, res = divmod(len(sendbuf), size)
        count = [ave + 1 if p < res else ave for p in range(size)]
        count = np.array(count)
        displ = [sum(count[:p]) for p in range(size)]
        displ = np.array(displ)
    else:
        sendbuf = None
        count = np.zeros(size, dtype=int)
        displ = np.zeros(size, dtype=int)
    comm.Bcast(count, root=0)
    comm.Bcast(displ, root=0)
    recvbuf = np.zeros(count[rank])
    comm.Scatterv([sendbuf, count, displ, MPI.DOUBLE], recvbuf, root=0)
    
    S_loc = np.array([0.0])
    comm.Bcast([S_loc, MPI.DOUBLE], root=0)    
    i_s = displ[rank]
    i_e = displ[rank]+count[rank]
    S_loc[0] = simpson_local_arr(y_arr, x_arr, i_s, i_e)
    comm.Barrier()
    
    if rank==0:
        S_tot = np.array([0.0])
    else:
        S_tot = None

    comm.Reduce(
        [S_loc, MPI.DOUBLE],
        [S_tot, MPI.DOUBLE],
        op=MPI.SUM,
        root=0)

    if rank==0:
        print("S = ", S_tot)
```
